In [ ]:
import cv2
import numpy as np
import json
from datetime import datetime
import matplotlib.pyplot as plt

class EnhancedMeasurementSystem:
    def __init__(self):
        self.quality_threshold = 0.8
        self.min_matches = 10
        self.measurement_uncertainty = 0.05  # 5% uncertainty
        
    def enhanced_feature_matching(self, template_gray, new_gray):
        """Enhanced feature matching with better filtering"""
        # SIFT feature detection
        sift = cv2.SIFT_create(nfeatures=500)
        kp1, des1 = sift.detectAndCompute(template_gray, None)
        kp2, des2 = sift.detectAndCompute(new_gray, None)
        
        if des1 is None or des2 is None:
            raise ValueError("Could not detect sufficient features")
        
        # FLANN matching
        FLANN_INDEX_KDTREE = 1
        index_params = dict(algorithm=FLANN_INDEX_KDTREE, trees=5)
        search_params = dict(checks=50)
        flann = cv2.FlannBasedMatcher(index_params, search_params)
        matches = flann.knnMatch(des1, des2, k=2)
        
        # Enhanced ratio test filtering
        good_matches = []
        for match_pair in matches:
            if len(match_pair) == 2:
                m, n = match_pair
                if m.distance < 0.7 * n.distance:
                    good_matches.append(m)
        
        # K-nearest neighbors validation
        good_matches = self.knn_filter(good_matches, kp1, kp2)
        
        return good_matches, kp1, kp2
    
    def knn_filter(self, matches, kp1, kp2, k=5):
        """K-nearest neighbors filtering for better matches"""
        if len(matches) < k:
            return matches
            
        # Calculate spatial consistency
        filtered_matches = []
        for i, match in enumerate(matches):
            pt1 = kp1[match.queryIdx].pt
            pt2 = kp2[match.trainIdx].pt
            
            # Find k nearest matches
            distances = []
            for j, other_match in enumerate(matches):
                if i != j:
                    other_pt1 = kp1[other_match.queryIdx].pt
                    other_pt2 = kp2[other_match.trainIdx].pt
                    
                    dist1 = np.linalg.norm(np.array(pt1) - np.array(other_pt1))
                    dist2 = np.linalg.norm(np.array(pt2) - np.array(other_pt2))
                    distances.append((j, abs(dist1 - dist2)))
            
            # Sort by spatial consistency
            distances.sort(key=lambda x: x[1])
            
            # Keep matches with good spatial consistency
            if len(distances) >= k-1:
                avg_inconsistency = np.mean([d[1] for d in distances[:k-1]])
                if avg_inconsistency < 50:  # Threshold for spatial consistency
                    filtered_matches.append(match)
        
        return filtered_matches
    
    def quality_check(self, homography, matches, kp1, kp2):
        """Quality check for measurement reliability"""
        if len(matches) < self.min_matches:
            return False, f"Insufficient matches: {len(matches)}"
        
        # Check homography condition
        det = np.linalg.det(homography[:2, :2])
        if abs(det) < 0.1 or abs(det) > 10:
            return False, f"Poor homography condition: {det}"
        
        # Check reprojection error
        src_pts = np.float32([kp1[m.queryIdx].pt for m in matches]).reshape(-1, 1, 2)
        dst_pts = np.float32([kp2[m.trainIdx].pt for m in matches]).reshape(-1, 1, 2)
        
        projected_pts = cv2.perspectiveTransform(src_pts, homography)
        errors = np.linalg.norm(projected_pts - dst_pts, axis=2).flatten()
        mean_error = np.mean(errors)
        
        if mean_error > 5.0:
            return False, f"High reprojection error: {mean_error:.2f}"
        
        return True, f"Quality OK: {len(matches)} matches, error: {mean_error:.2f}"
    
    def pixel_to_physical_conversion(self, pixel_length, dx, dy, ref_horizontal, ref_vertical):
        """Enhanced pixel to physical unit conversion with uncertainty"""
        if ref_horizontal <= 0 or ref_vertical <= 0:
            raise ValueError("Invalid reference measurements")
        
        # Calculate components
        horizontal_component = abs(dx) / ref_horizontal
        vertical_component = abs(dy) / ref_vertical
        
        # Physical length
        length_cm = np.sqrt(horizontal_component**2 + vertical_component**2)
        
        # Calculate uncertainty
        h_uncertainty = self.measurement_uncertainty * horizontal_component
        v_uncertainty = self.measurement_uncertainty * vertical_component
        total_uncertainty = np.sqrt(h_uncertainty**2 + v_uncertainty**2)
        
        return length_cm, total_uncertainty
    
    def generate_measurement_report(self, measurements, image_path, quality_info):
        """Generate structured measurement report"""
        timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        
        report = {
            "metadata": {
                "timestamp": timestamp,
                "image_path": image_path,
                "quality_score": quality_info,
                "measurement_count": len(measurements)
            },
            "measurements": [],
            "statistics": {
                "total_measurements": len(measurements),
                "avg_length": 0,
                "min_length": float('inf'),
                "max_length": 0
            }
        }
        
        lengths = []
        for i, measurement in enumerate(measurements):
            length = measurement['length_cm']
            uncertainty = measurement.get('uncertainty', 0)
            lengths.append(length)
            
            report["measurements"].append({
                "id": i + 1,
                "label": measurement['label'],
                "length_cm": round(length, 3),
                "length_mm": round(length * 10, 2),
                "length_inches": round(length / 2.54, 3),
                "uncertainty_cm": round(uncertainty, 4),
                "confidence": "High" if uncertainty < 0.05 else "Medium" if uncertainty < 0.1 else "Low",
                "angle_degrees": measurement['angle_degrees'],
                "coordinates": {
                    "point1": measurement['point1'],
                    "point2": measurement['point2']
                }
            })
        
        # Calculate statistics
        if lengths:
            report["statistics"]["avg_length"] = round(np.mean(lengths), 3)
            report["statistics"]["min_length"] = round(np.min(lengths), 3)
            report["statistics"]["max_length"] = round(np.max(lengths), 3)
            report["statistics"]["std_deviation"] = round(np.std(lengths), 3)
        
        return report
    
    def enhanced_match_and_measure(self, new_image_path, template_image_path, template_json_path):
        """Enhanced matching and measurement with all improvements"""
        try:
            # Read images
            new_img = cv2.imread(new_image_path)
            template_img = cv2.imread(template_image_path)
            
            if new_img is None or template_img is None:
                raise ValueError("Could not load images")
            
            # Convert to grayscale
            new_gray = cv2.cvtColor(new_img, cv2.COLOR_BGR2GRAY)
            template_gray = cv2.cvtColor(template_img, cv2.COLOR_BGR2GRAY)
            
            # Load template data
            with open(template_json_path, 'r') as f:
                template_data = json.load(f)
            
            # Enhanced feature matching
            good_matches, kp1, kp2 = self.enhanced_feature_matching(template_gray, new_gray)
            
            # Get matched points for homography
            src_pts = np.float32([kp1[m.queryIdx].pt for m in good_matches]).reshape(-1, 1, 2)
            dst_pts = np.float32([kp2[m.trainIdx].pt for m in good_matches]).reshape(-1, 1, 2)
            
            # Find homography with RANSAC
            homography, mask = cv2.findHomography(
                src_pts, dst_pts, 
                cv2.RANSAC, 
                ransacReprojThreshold=5.0,
                confidence=0.99
            )
            
            if homography is None:
                raise ValueError("Could not compute homography")
            
            # Quality check
            quality_ok, quality_info = self.quality_check(homography, good_matches, kp1, kp2)
            if not quality_ok:
                print(f"Warning: {quality_info}")
            
            # Process measurements
            result_img = new_img.copy()
            measurements = []
            ref_horizontal = template_data['reference']['horizontal']
            ref_vertical = template_data['reference']['vertical']
            
            for measure in template_data['measurements']:
                # Transform points
                pt1 = np.float32([[measure['point1']['x'], measure['point1']['y']]])
                pt2 = np.float32([[measure['point2']['x'], measure['point2']['y']]])
                
                transformed_pt1 = cv2.perspectiveTransform(pt1.reshape(-1, 1, 2), homography)
                transformed_pt2 = cv2.perspectiveTransform(pt2.reshape(-1, 1, 2), homography)
                
                pt1_new = tuple(map(int, transformed_pt1[0][0]))
                pt2_new = tuple(map(int, transformed_pt2[0][0]))
                
                # Calculate measurements with uncertainty
                dx = pt2_new[0] - pt1_new[0]
                dy = pt2_new[1] - pt1_new[1]
                pixel_length = np.sqrt(dx**2 + dy**2)
                
                length_cm, uncertainty = self.pixel_to_physical_conversion(
                    pixel_length, dx, dy, ref_horizontal, ref_vertical
                )
                
                # Calculate angle
                angle = np.degrees(np.arctan2(dy, dx))
                if angle < 0:
                    angle += 360
                
                # Store measurement
                measurements.append({
                    'label': measure['label'],
                    'point1': pt1_new,
                    'point2': pt2_new,
                    'length_cm': length_cm,
                    'uncertainty': uncertainty,
                    'angle_degrees': angle
                })
                
                # Draw on image
                cv2.circle(result_img, pt1_new, 5, (255, 0, 0), -1)
                cv2.circle(result_img, pt2_new, 5, (255, 0, 0), -1)
                cv2.line(result_img, pt1_new, pt2_new, (0, 255, 0), 2)
                
                # Add enhanced labels
                mid_point = ((pt1_new[0] + pt2_new[0])//2, (pt1_new[1] + pt2_new[1])//2)
                cv2.putText(result_img, measure['label'], 
                           (mid_point[0], mid_point[1]-10), 
                           cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 0, 0), 2)
                cv2.putText(result_img, f'{length_cm:.2f}±{uncertainty:.3f}cm', 
                           (mid_point[0], mid_point[1]+20),
                           cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2)
            
            # Generate comprehensive report
            report = self.generate_measurement_report(measurements, new_image_path, quality_info)
            
            # Save results
            timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
            cv2.imwrite(f'enhanced_measurement_{timestamp}.png', result_img)
            
            with open(f'enhanced_report_{timestamp}.json', 'w') as f:
                json.dump(report, f, indent=4)
            
            return result_img, measurements, report
            
        except Exception as e:
            print(f"Error in measurement process: {str(e)}")
            return None, None, None

# Usage example
if __name__ == "__main__":
    system = EnhancedMeasurementSystem()
    # new_image = 
    # template_image = 
    # template_json = 
    
    result_img, measurements, report = system.enhanced_match_and_measure(
        new_image, template_image, template_json
    )
    
    if report:
        print("Measurement Report:")
        print(f"Total measurements: {report['statistics']['total_measurements']}")
        print(f"Average length: {report['statistics']['avg_length']} cm")
        for measurement in report['measurements']:
            print(f"{measurement['label']}: {measurement['length_cm']} ± {measurement['uncertainty_cm']} cm")